In [16]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
from copy import deepcopy

import numpy as np
import pandas as pd
from ordinor.utils.validation import check_convert_input_log

from ordinor.io import read_disco_csv
from ordinor.execution_context.rule_based import ODTMiner


In [17]:

# input log
#fn_log = sys.argv[1]

#el = read_disco_csv(fn_log)
#el = pd.read_csv("bpic15_amended_typed.csv")
el = pd.read_csv("./data/processed/bpic15.csv")
el.head()

,Unnamed: 0,Case ID,activityNameEN,org:resource,Complete Timestamp,ct:last_phase,case_parts,action_code,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm
0,0,10009138,register submission date request,9264148,2014-04-10 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_010,muni-1,Bouw,01_HOOFD_0,Apr,Day_10,Thu,PM
1,1,10009138,OLO messaging active,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_011,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
2,2,10009138,send confirmation receipt,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_020,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
3,5,10009138,create procedure confirmation,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_065_0,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
4,6,10009138,create subcases completeness,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_110_0,muni-1,Bouw,01_HOOFD_1,Apr,Day_13,Sun,PM


In [18]:
#df = el.rename(columns={
#            # Resource-related
#            'Case ID' : 'case:concept:name',
#            'Complete Timestamp': 'time:timestamp',
#            "Resource": "org:resource",
#            "Activity": ":conceptname",
#        })
df = el.rename(columns={
            # Resource-related
            'Case ID' : 'case:concept:name',
            'Complete Timestamp': 'time:timestamp',
            "Resource": "org:resource",
            "action_code": "concept:name",
        })
df.head()

,Unnamed: 0,case:concept:name,activityNameEN,org:resource,time:timestamp,ct:last_phase,case_parts,concept:name,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm
0,0,10009138,register submission date request,9264148,2014-04-10 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_010,muni-1,Bouw,01_HOOFD_0,Apr,Day_10,Thu,PM
1,1,10009138,OLO messaging active,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_011,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
2,2,10009138,send confirmation receipt,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_020,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
3,5,10009138,create procedure confirmation,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_065_0,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
4,6,10009138,create subcases completeness,9264148,2014-04-13 22:00:00+00:00,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_110_0,muni-1,Bouw,01_HOOFD_1,Apr,Day_13,Sun,PM


In [19]:
test = check_convert_input_log(df)

In [20]:

# specification

spec = {
    'type_def_attrs': {
        # BPIC15
        'ct:permit_type': {'attr_type': 'categorical', 'attr_dim': 'CT'},
        'at:phase': {'attr_type': 'categorical', 'attr_dim': 'AT'},
        'tt:weekday': {'attr_type': 'categorical', 'attr_dim': 'TT'}, 
        'tt:ampm': {'attr_type': 'categorical', 'attr_dim': 'TT'},
    }
}


df = df[df['case:concept:name'] != 4020737]

df_copy=df


miner = ODTMiner(df, spec, max_height=12, trace_history=True)



C:\Users\lasse\APM\No-time-to-dice_group_project_IE692\OrdinoR-dev\ordinor\execution_context\rule_based\decision_tree.py:42: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self._log = el[included_cols]
C:\Users\lasse\APM\No-time-to-dice_group_project_IE692\OrdinoR-dev\ordinor\execution_context\rule_based\decision_tree.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._log[const.RESOURCE] = self._log[const.RESOURCE].astype('category')
C:\Users\lasse\APM\No-time-to-dice_group_project_IE692\OrdinoR-dev\ordinor\execution_context\rule_based\decision_tree.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Decision tree initialized with an empty root node
	Dis. = 0.000000, Imp. = 1.000000, Harmonic Mean. = 0.000000, *** Tree has 1 leaf node(s). ***
Start to fit decision tree with max. height = 12
Tree grows by splitting all current leaf nodes on `at:phase`
	Dis. = 0.000155, Imp. = 0.999941, Harmonic Mean. = 0.000310, *** Tree has 2 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `tt:weekday`
	Dis. = 0.000846, Imp. = 0.999770, Harmonic Mean. = 0.001691, *** Tree has 3 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`
	Dis. = 0.003602, Imp. = 0.998144, Harmonic Mean. = 0.007179, *** Tree has 4 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`
	Dis. = 0.035858, Imp. = 0.990776, Harmonic Mean. = 0.069211, *** Tree has 5 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `tt:weekday`
	Dis. = 0.097113, Imp. = 0.984761, Harmonic Mean. = 0.176792, *** Tree has 8 leaf node(s). ***
Tree grows by splittin

In [21]:
%%script false --no-raise-error
def insert_only_new_atomicRule_to_list(list, atomicRule):
    for a_rule in list:
        if a_rule==atomicRule:## __eq__ method in AtomicRule class seems reasonable
            return
    else:
        list.append(atomicRule)
def check_attr_already_exists_in_list(list,exRule):
    for i, _ in enumerate(list):
        if exRule.is_same_attr(list[i][0]): # take index 0(first rule) because all rules in that index have the same attr
            return i
    return -1

Couldn't find program: 'false'


In [22]:
%%script false --no-raise-error
#extract all rules from leafes into one list
all = []
for key, node in miner._leaves.items(): 
    for i, _ in enumerate(node.at_rule.ars):
        insert_only_new_atomicRule_to_list(all,node.at_rule.ars[i])

    for i, _ in enumerate(node.ct_rule.ars):
        insert_only_new_atomicRule_to_list(all,node.ct_rule.ars[i])

    for i, _ in enumerate(node.tt_rule.ars):
        insert_only_new_atomicRule_to_list(all,node.tt_rule.ars[i])

#create list of atomicrules indexed by their attr
#der umweg über die list wird gegangen, da im dict nur noch der name des attr steht und daher die is_same_attr(self, other) der AtomicRule Klasse nicht mehr genutzt werden könnte --> im unwahrscheinlichen fall dass 2 spalten gleich heißen würden hier Probleme reduziert werden
all_disaggregated = []
for rule in all:
    # check if list has an index with rules that are the same as the current rule
    pos = check_attr_already_exists_in_list(all_disaggregated,rule)
    if pos == -1:
        l = []
        l.append(rule)
        all_disaggregated.append(l)
    else:
        all_disaggregated[pos].append(rule)

#create dictionary out of list; key is attr, value is list of attr_vals(which is a frozen set)
all_disaggregated_dict = {}

for i, _ in enumerate(all_disaggregated):
    all_disaggregated_dict[all_disaggregated[i][0].attr] = all_disaggregated[i]#any item in all_disaggregated[i] works; [0] is always available

print(all)

Couldn't find program: 'false'


In [23]:
%%script false --no-raise-error
def get_corresponding_set_as_string(value,list_of_AtomicRules):
    for atomicRule in list_of_AtomicRules:
        if value in atomicRule.attr_vals:
            
            return "-".join(atomicRule.attr_vals)
    return None

Couldn't find program: 'false'


In [24]:
%%script false --no-raise-error
df = pd.read_csv("./data/processed/bpic15.csv")
prefix="RES_"

#initialize all result columns with None --> coulumns were no rules were learned on will still be None
for attr_spec,_ in spec["type_def_attrs"].items():
    df[prefix+attr_spec] = None
    

# add new columns to df that represent the learned rules
for key, list_of_sets in all_disaggregated_dict.items():
    if key: #check if key is None
        if not key in df.columns: 
            raise Exception(f"{key=} not found in dataframe")
        df[prefix+str(key)] = df[key].apply(lambda x: get_corresponding_set_as_string(x,list_of_sets))
df.head()




#combine columns to CO columnn
for col in df.columns:
    if col.startswith(prefix):
        if prefix+"CO" not in df.columns:
            df[prefix+"CO"] = df[col].astype(str)
        else:   
            df[prefix+"CO"] = df[prefix+"CO"] + " | " + df[col].astype(str)

#write df to csv
df.to_csv('output.csv', index=False)


Couldn't find program: 'false'


In [25]:
%%script false --no-raise-error
miner.print_tree()


Couldn't find program: 'false'


In [26]:
df = pd.read_csv("./data/processed/bpic15.csv")

# get different rules
l_rules_ct, l_rules_at, l_rules_tt = miner._parse_rules_from_leaves(miner._leaves)

# label ATs
dfs = [r.apply(df, index_only=False).assign(AT_rule=r) for r in l_rules_at]
df = pd.concat(dfs)

# label CTs
dfs = [r.apply(df, index_only=False).assign(CT_rule=r) for r in l_rules_ct]
df = pd.concat(dfs)

# label TTs
dfs = [r.apply(df, index_only=False).assign(TT_rule=r) for r in l_rules_tt]
df = pd.concat(dfs)

In [39]:
%%script false --no-raise-error
# other possibility to join rules: 
# leaves contain Nodes which contain the corresponding column ids of the events in the event log
for _, n in miner._leaves.items():
    print("*"*60)
    print(n.event_ids)

************************************************************
Int64Index([13044, 13055, 60981, 60984, 73727, 73736, 76104, 76344, 76798,
            77027, 77029, 77628, 77638, 78573, 78584],
           dtype='int64')
************************************************************
Int64Index([  6346,   6401,   7119,   7136,   7143,   7954,   8456,   9125,
              9216,   9265,
            ...
            171706, 171814, 171850, 171886, 172072, 172108, 172217, 172253,
            172323, 173062],
           dtype='int64', length=543)
************************************************************
Int64Index([  6672,   6708,   6744,   7120,   7257,   9989,  12199,  16997,
             58813,  58889,  58927,  58965,  59003,  59138,  59174,  59263,
             59343,  59389,  59432,  59513,  59688,  59874,  61182,  61892,
             64839,  65039,  65271,  65854, 125297, 160359, 164484, 164582,
            164660, 164917, 165546, 165586, 165629, 166307, 166590, 166629,
            166710

In [27]:
# add CO_rule as combination of AT CT and TT
from ordinor.execution_context.rule_based.Rule import Rule
df['CO_rule'] = df.apply(lambda x: Rule(x['AT_rule'].ars+x['CT_rule'].ars+x['TT_rule'].ars), axis=1)

In [28]:
df['CO_rule'][0]

(⊥ (null)) ∧ (`at:phase` ∈ {'01_HOOFD_0', '01_HOOFD_1', '01_HOOFD_4', '01_HOOFD_5'}) ∧ (`tt:weekday` ∈ {'Mon', 'Sun', 'Thu', 'Tue', 'Wed'})

In [29]:
df.to_csv('output.csv', index=False)
df.head()

,Unnamed: 0,Case ID,activityNameEN,org:resource,Complete Timestamp,ct:last_phase,case_parts,action_code,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm,AT_rule,CT_rule,TT_rule,CO_rule
13044,16631,3462821,enter senddate continuation,560589,2011-06-21 22:00:00+00:00,Procedure afgebroken,Bouw,01_HOOFD_600_2,muni-1,Bouw,01_HOOFD_6,Jun,Day_21,Tue,PM,(`at:phase` ∈ {'01_HOOFD_6'}),⊥ (null),"(`tt:weekday` ∈ {'Mon', 'Sun', 'Thu', 'Tue', '...",(⊥ (null)) ∧ (`at:phase` ∈ {'01_HOOFD_6'}) ∧ (...
13055,16648,3462821,send continuation letter,560589,2011-06-22 09:52:04+00:00,Procedure afgebroken,Bouw,01_HOOFD_600_1,muni-1,Bouw,01_HOOFD_6,Jun,Day_22,Wed,AM,(`at:phase` ∈ {'01_HOOFD_6'}),⊥ (null),"(`tt:weekday` ∈ {'Mon', 'Sun', 'Thu', 'Tue', '...",(⊥ (null)) ∧ (`at:phase` ∈ {'01_HOOFD_6'}) ∧ (...
60981,81947,3657921,enter senddate continuation,560458,2011-05-02 22:00:00+00:00,Zaak afgehandeld,Bouw,01_HOOFD_600_2,muni-2,Bouw,01_HOOFD_6,May,Day_2,Mon,PM,(`at:phase` ∈ {'01_HOOFD_6'}),⊥ (null),"(`tt:weekday` ∈ {'Mon', 'Sun', 'Thu', 'Tue', '...",(⊥ (null)) ∧ (`at:phase` ∈ {'01_HOOFD_6'}) ∧ (...
60984,81950,3657921,send continuation letter,560458,2011-05-03 15:34:09+00:00,Zaak afgehandeld,Bouw,01_HOOFD_600_1,muni-2,Bouw,01_HOOFD_6,May,Day_3,Tue,PM,(`at:phase` ∈ {'01_HOOFD_6'}),⊥ (null),"(`tt:weekday` ∈ {'Mon', 'Sun', 'Thu', 'Tue', '...",(⊥ (null)) ∧ (`at:phase` ∈ {'01_HOOFD_6'}) ∧ (...
73727,99138,3197901,enter senddate continuation,2013365,2011-03-15 23:00:00+00:00,Advies bekend,Bouw,01_HOOFD_600_2,muni-3,Bouw,01_HOOFD_6,Mar,Day_15,Tue,PM,(`at:phase` ∈ {'01_HOOFD_6'}),⊥ (null),"(`tt:weekday` ∈ {'Mon', 'Sun', 'Thu', 'Tue', '...",(⊥ (null)) ∧ (`at:phase` ∈ {'01_HOOFD_6'}) ∧ (...
